# Lesson 1: Your first agent with Amazon Bedrock

## Preparation 
<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> and other files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

In [1]:
# Before you start, please run the following code to set up your environment.
# This code will reset the environment (if needed) and prepare the resources for the lesson.
# It does this by quickly running through all the code from the previous lessons.

!sh ./ro_shared_data/reset.sh

import os

roleArn = os.environ['BEDROCKAGENTROLE']

Resetting environment (if nessesary)
Agent reset process completed.
Lambda reset process completed.
Guardrail reset process completed.
Environment reset complete.


## Start of the lesson

In [2]:
import boto3

In [3]:
bedrock_agent = boto3.client(service_name='bedrock-agent', region_name='us-west-2')

In [4]:
create_agent_response = bedrock_agent.create_agent(
    agentName='mugs-customer-support-agent',
    foundationModel='anthropic.claude-3-haiku-20240307-v1:0',
    instruction="""You are an advanced AI agent acting as a front line customer support agent.""",
    agentResourceRoleArn=roleArn
)

In [5]:
create_agent_response

{'ResponseMetadata': {'RequestId': 'fb0ca41c-ed65-4bc4-81d7-f926328fae54',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Fri, 30 May 2025 16:48:06 GMT',
   'content-type': 'application/json',
   'content-length': '624',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'fb0ca41c-ed65-4bc4-81d7-f926328fae54',
   'x-amz-apigw-id': 'LZAi9HXIPHcEiVA=',
   'x-amzn-trace-id': 'Root=1-6839e145-7d90e54871b144b814a40851'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-west-2:851725191978:agent/C8ME56KMFR',
  'agentId': 'C8ME56KMFR',
  'agentName': 'mugs-customer-support-agent',
  'agentResourceRoleArn': 'arn:aws:iam::851725191978:role/c135316a3428926l10466193t1w8517251-BedrockAgentRole-QzVkHS1cJV2J',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2025, 5, 30, 16, 48, 5, 811085, tzinfo=tzlocal()),
  'foundationModel': 'anthropic.claude-3-haiku-20240307-v1:0',
  'idleSessionTTLInSeconds': 600,
  'instruction': 'You are an advanced AI agent acting as a 

In [6]:
agentId = create_agent_response['agent']['agentId']

In [7]:
from helper import *

In [8]:
wait_for_agent_status(
    agentId=agentId, 
    targetStatus='NOT_PREPARED'
)

Waiting for agent status of 'NOT_PREPARED'...
Agent status: NOT_PREPARED
Agent reached 'NOT_PREPARED' status.


In [9]:
bedrock_agent.prepare_agent(
    agentId=agentId
)

{'ResponseMetadata': {'RequestId': 'cc28fd04-327d-4134-b1c1-96d21d4b9d33',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Fri, 30 May 2025 16:48:10 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'cc28fd04-327d-4134-b1c1-96d21d4b9d33',
   'x-amz-apigw-id': 'LZAjpEUQvHcEJZA=',
   'x-amzn-trace-id': 'Root=1-6839e14a-13442f42705578c52ab357b4'},
  'RetryAttempts': 0},
 'agentId': 'C8ME56KMFR',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2025, 5, 30, 16, 48, 10, 400919, tzinfo=tzlocal())}

In [10]:
wait_for_agent_status(
    agentId=agentId, 
    targetStatus='PREPARED'
)

Waiting for agent status of 'PREPARED'...
Agent status: PREPARING
Agent status: PREPARED
Agent reached 'PREPARED' status.


In [11]:
create_agent_alias_response = bedrock_agent.create_agent_alias(
    agentId=agentId,
    agentAliasName='MyAgentAlias',
)

agentAliasId = create_agent_alias_response['agentAlias']['agentAliasId']

wait_for_agent_alias_status(
    agentId=agentId,
    agentAliasId=agentAliasId,
    targetStatus='PREPARED'
)

Waiting for agent alias status of 'PREPARED'...
Agent alias status: CREATING
Agent alias status: CREATING
Agent alias status: PREPARED
Agent alias reached status 'PREPARED'


In [12]:
bedrock_agent_runtime = boto3.client(service_name='bedrock-agent-runtime', region_name='us-west-2')

In [13]:
import uuid

In [14]:
message = "Hello, I bought a mug from your store yesterday, and it broke. I want to return it."

sessionId = str(uuid.uuid4())

invoke_agent_response = bedrock_agent_runtime.invoke_agent(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    endSession=False,
    enableTrace=True,
)

In [15]:
event_stream = invoke_agent_response["completion"]

In [16]:
for event in event_stream:
    print(event)

{'trace': {'agentAliasId': 'UUQD1NN3PA', 'agentId': 'C8ME56KMFR', 'agentVersion': '1', 'sessionId': 'ddf232c3-1012-43d5-93df-3a69e99938a9', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'inferenceConfiguration': {'maximumLength': 2048, 'stopSequences': ['</invoke>', '</answer>', '</error>'], 'temperature': 0.0, 'topK': 250, 'topP': 1.0}, 'text': '{"system":" You are an advanced AI agent acting as a front line customer support agent. You have been provided with a set of functions to answer the user\'s question. You must call the functions in the format below: <function_calls>   <invoke>     <tool_name>$TOOL_NAME</tool_name>     <parameters>       <$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>       ...     </parameters>   </invoke> </function_calls> Here are the functions available: <functions>    </functions> You will ALWAYS follow the below guidelines when you are answering a question: <guidelines> - Think through the user\'s question, extract all data from the questi

{'trace': {'agentAliasId': 'UUQD1NN3PA', 'agentId': 'C8ME56KMFR', 'agentVersion': '1', 'sessionId': 'ddf232c3-1012-43d5-93df-3a69e99938a9', 'trace': {'orchestrationTrace': {'modelInvocationOutput': {'metadata': {'usage': {'inputTokens': 877, 'outputTokens': 51}}, 'rawResponse': {'content': 'I apologize, but I seem to be having trouble accessing your order details without more information. Could you please provide the order ID or the date you made the purchase? That will help me look up the details and process your return request.'}, 'traceId': '365be7ac-d1ce-46d9-90ca-ad8a3e2a8dd0-2'}}}}}
{'trace': {'agentAliasId': 'UUQD1NN3PA', 'agentId': 'C8ME56KMFR', 'agentVersion': '1', 'sessionId': 'ddf232c3-1012-43d5-93df-3a69e99938a9', 'trace': {'orchestrationTrace': {'observation': {'finalResponse': {'text': 'I apologize, but I seem to be having trouble accessing your order details without more information. Could you please provide the order ID or the date you made the purchase? That will help 

In [17]:
message = "Hello, I bought a mug from your store yesterday, and it broke. I want to return it."

sessionId = str(uuid.uuid4())

In [18]:
invoke_agent_and_print(
    agentAliasId=agentAliasId,
    agentId=agentId,
    sessionId=sessionId,
    inputText=message,
    enableTrace=True,
)

User: Hello, I bought a mug from your store yesterday, and it broke. I want
to return it.

Agent: 
Agent's thought process:
  Okay, let's go through the steps to handle this return request:

Agent's thought process:
  Apologies, let me try that again in the correct format:

Agent's thought process:
  I apologize, it seems there was an issue with my previous function
  call. Let me try a different approach to get the return policy
  information.

Agent's thought process:
  I apologize, it seems there is an issue with the function call
  format. Let me try to gather the necessary information to assist you
  with your return request.

Observation:
  Type: FINISH

Final response:
  1. Gather the mug and your proof of purchase (such as the receipt).
  2. Contact our customer service team at [phone number] or [email] to
  initiate the return process. 3. Provide the customer service
  representative with the details of your purchase and the reason for
  the return (the mug being broken). 4. F